references
https://python.langchain.com/docs/integrations/llms/ollama/

https://api.python.langchain.com/en/latest/graph_transformers/langchain_experimental.graph_transformers.llm.LLMGraphTransformer.html

https://python.langchain.com/docs/how_to/document_loader_json/
https://thesai.org/Downloads/Volume10No10/Paper_30-JsonToOnto_Building_Owl2_Ontologies.pdf

owl to neo4j to graphRAG
https://neo4j.com/blog/using-owl-with-neo4j/?utm_source=GSearch&utm_medium=PaidSearch&utm_campaign=Evergreen&utm_content=AMS-Search-SEMCE-DSA-None-SEM-SEM-NonABM&utm_term=&utm_adgroup=DSA&gad_source=1&gclid=CjwKCAiAudG5BhAREiwAWMlSjFNimXejezc7i1kEQlYU_gi_es4y2GsJU0q_egaOoWvEB3w8YzP3SRoC-mAQAvD_BwE

https://neo4j.com/labs/neosemantics/4.0/importing-ontologies/

https://neo4j.com/developer-blog/neo4j-graphrag-workflow-langchain-langgraph/

they used microsoft graphRAG
https://medium.com/@ianormy/microsoft-graphrag-with-an-rdf-knowledge-graph-part-1-00a354afdb09

https://medium.com/@jinglemind.dev/mastering-advanced-rag-methods-graphrag-with-neo4j-implementation-with-langchain-42b8f1d05246

sudo systemctl start neo4j
systemctl {start|stop|restart} neo4j

neo4j=1.5.25.1

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# from llama_index.embeddings.langchain import LangchainEmbedding

# MODEL_NAME = 'all-MiniLM-L6-v2'

# EmbeddingModel(model_name = MODEL_NAME).get_model()

In [3]:
# import networkx as nx
# from langchain.chains import GraphQAChain
# from langchain_core.documents import Document
# from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

In [64]:
url = "bolt://localhost:7687"
username = "neo4j"
password= "neo4j"

driver = GraphDatabase.driver(url, auth=(username, password))

dont touch, adds data to database and configures it.

In [65]:
def create_constraint(driver):
    # Create a session
    with driver.session() as session:
        # Run the updated Cypher query to create the constraint
        cypher_query = """CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE;"""
        session.run(cypher_query)
        print("Constraint created successfully.")

# Call the function to create the constraint
create_constraint(driver)

# old constraint "CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE;"

Constraint created successfully.


In [25]:
import rdflib
from collections import defaultdict

def extract_multivalued_properties(owl_file):
     # Parse the RDF graph from the given OWL file
    graph = rdflib.Graph()
    graph.parse(owl_file, format='xml')
    
    # Dictionary to store property values (property -> [(subject, object)])
    property_values = defaultdict(list)
    
    # Iterate through the RDF graph and collect property-value pairs
    for subj, pred, obj in graph:
        property_values[str(pred)].append((str(subj), str(obj)))
        
    # Dictionary to store multivalued properties (property -> [subject])
    multivalued_properties = []
    
    # Check for multivalued properties (where a property has multiple unique objects for the same subject)
    for prop, values in property_values.items():
        # Skip 'rdf:type' property from being processed
        if prop == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type':
            continue
        
        subjects = defaultdict(set)
        
        # Group objects by subject
        for subj, obj in values:
            subjects[subj].add(obj)
        
        # If a subject has more than one unique object, it's a multivalued property
        for subj, objs in subjects.items():
            if len(objs) > 1:
                multivalued_properties.append(subj)
    
    # Format the output as a comma-separated string of URIs in the required format
    formatted_properties = ", ".join([f"'{uri}'" for uri in multivalued_properties])
    
    return formatted_properties

owl_file = "./data/owl/annett-o-0.1.owl"

multivalued_properties = extract_multivalued_properties(owl_file)

print(multivalued_properties)
    

'http://w3id.org/annett-o/AAE_Label_GAN', 'http://w3id.org/annett-o/AAE_Encoder_Style', 'http://w3id.org/annett-o/AAE_AE', 'http://w3id.org/annett-o/GAN_net', 'http://w3id.org/annett-o/GAN_Discriminator', 'http://w3id.org/annett-o/GAN_Generator', 'http://w3id.org/annett-o/AAE_Encoder_Label', 'http://w3id.org/annett-o/AAE_Style_Discriminator', 'http://w3id.org/annett-o/AAE_Label_Discriminator', 'http://w3id.org/annett-o/AAE_Style_GAN', 'http://w3id.org/annett-o/simple_classification_net', 'http://w3id.org/annett-o/AAE_AE_Concat', 'http://w3id.org/annett-o/InputLayer', 'http://w3id.org/annett-o/ConvolutionLayer', 'http://w3id.org/annett-o/EvaluationResult', 'http://w3id.org/annett-o/NetworkSpecific', 'http://w3id.org/annett-o/AggregationLayer', 'http://w3id.org/annett-o/TrainingSession', 'http://w3id.org/annett-o/TrainingLoop', 'http://w3id.org/annett-o/ModificationLayer', 'http://w3id.org/annett-o/ActivationLayer', 'http://w3id.org/annett-o/OutputLayer', 'http://w3id.org/annett-o/Iterat

In [66]:
def queryNeo4j(driver, query):
    """Runs a single Cypher query."""
    with driver.session() as session:
        try:
            session.run(query)
            # print("Query executed successfully.")
        except Exception as e:
            print(f"Error executing query: {e}")

In [ ]:
# config = "CALL n10s.graphconfig.init({ handleVocabUris: \"IGNORE\", keepLangTag: true, handleMultival: \"ARRAY\", handleRDFTypes: \"LABELS_AND_NODES\", " + f"multivalPropList : [{multivalued_properties}]" + "});"
# queryNeo4j(driver, config)



In [67]:
queryNeo4j(driver, "CALL n10s.graphconfig.init({ handleVocabUris: \"SHORTEN\", keepLangTag: false, handleMultival: \"ARRAY\"});")

In [68]:
queryNeo4j(driver, "CALL n10s.rdf.import.fetch(\"https://raw.githubusercontent.com/iaklampanos/annett-o/refs/heads/master/annett-o-0.1.owl\",\"RDF/XML\");")

# queryNeo4j(driver, "CALL n10s.rdf.import.fetch(\"https://raw.githubusercontent.com/iaklampanos/annett-o/refs/heads/master/annett-o-0.1.owl\",\"RDF/XML\", {typesToLabels: false});")

In [62]:
queryNeo4j(driver , "MATCH(n) DETACH DELETE n;")

In [63]:
queryNeo4j(driver , "DROP CONSTRAINT n10s_unique_uri")

In [61]:
queryNeo4j(driver , "CALL n10s.nsprefixes.add(\"http://w3id.org/annett-o/\")")

print(queryNeo4j(driver , "CALL n10s.nsprefixes.list()"))

Error executing query: {code: Neo.ClientError.Statement.SyntaxError} {message: Procedure call does not provide the required number of arguments: got 1 expected at least 2 (total: 2, 0 of which have default values).

Procedure n10s.nsprefixes.add has signature: n10s.nsprefixes.add(prefix :: STRING, ns :: STRING) :: prefix :: STRING, namespace :: STRING
meaning that it expects at least 2 arguments of types STRING, STRING
Description: Adds namespace - prefix pair definition (line 1, column 1 (offset: 0))
"CALL n10s.nsprefixes.add("http://w3id.org/annett-o/")"
 ^}
None


In [ ]:
# def findGAN():
#     session = driver.session()
#     query = """
#     MATCH (n {uri: "http://w3id.org/annett-o/simple_classification_net"})
#     RETURN n
#     """
#     widget = GraphWidget(graph = session.run(query).graph())
#     widget.node_label_mapping = 'name'
#     widget.set_sidebar(start_with='Data')
#     return widget

# findGAN()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [69]:
def showGraph():
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (n)-[r]->(m) RETURN n, r, m").graph())
    widget.node_label_mapping = 'label'
    widget.hierarchic_layout()
    widget.set_sidebar(start_with='Data')
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [5]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model = "llama3.1:8b-instruct-fp16") 

llm_transformer = LLMGraphTransformer(llm=llm)


In [ ]:
#makes a wrapper instance of Neo4j graph then populates it with the data in the database
ontology_graph = Neo4jGraph(url=url, username=username, password=password)
# ontology_graph.query("MATCH (n)-[r]->(m) RETURN n, r, m") checking if it has the data

[{'n': {'name': 'AggregationLayer',
   'comment': ['An aggregation layer provides some static (untrainable) aggregation of its inputs, e.g. through feaure concatenation.'],
   'uri': 'http://w3id.org/annett-o/AggregationLayer'},
  'r': ({'name': 'AggregationLayer',
    'comment': ['An aggregation layer provides some static (untrainable) aggregation of its inputs, e.g. through feaure concatenation.'],
    'uri': 'http://w3id.org/annett-o/AggregationLayer'},
   'SCO_RESTRICTION',
   {'name': 'Layer',
    'comment': ['An ANN layer. Layers can be IO or hidden. Each network described using the ontology must define at least an input layer.'],
    'uri': 'http://w3id.org/annett-o/Layer'}),
  'm': {'name': 'Layer',
   'comment': ['An ANN layer. Layers can be IO or hidden. Each network described using the ontology must define at least an input layer.'],
   'uri': 'http://w3id.org/annett-o/Layer'}},
 {'n': {'name': 'AggregationLayer',
   'comment': ['An aggregation layer provides some static (un

adversial autoencoder, GAN, simple classification

notes:

have the llm make a dispute between two networks and keep doing it with additional instances to give a similarity score.

need to build a query chain for the llm to reference the graph and answer questions for it.

In [ ]:
class Entities(BaseModel):
    """"""

We'll be doing the paper context and embeddings later, this is work in progress

In [27]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model = "llama3.1:8b-instruct-fp16") 

llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_core.documents import Document

pdf_loader = PyPDFLoader("papers/test.pdf")
documents = pdf_loader.load()

llm_transformer = LLMGraphTransformer(llm=llm, strict_mode=True)
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")


storing graph documents

In [ ]:
graph = Neo4jGraph()
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"
def showGraph(cypher: str = default_cypher):
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
    )
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

In [ ]:
from langchain_core.documents import Document

text = """ 
class AlexNet(nn.Module):
    
    Neural network model consisting of layers propsed by AlexNet paper.
    
    def __init__(self, num_classes=1000):
        
        Define and allocate layers for this neural net.

        Args:
            num_classes (int): number of classes to predict with this model
        
        super().__init__()
        # input size should be : (b x 3 x 227 x 227)
        # The image in the original paper states that width and height are 224 pixels, but
        # the dimensions after first convolution layer do not lead to 55 x 55.
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 55 x 55)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  # (b x 256 x 13 x 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
        )
        # classifier is just a name for linear layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        self.init_bias()  # initialize bias 
"""

In [9]:
documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm, strict_mode=True)
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='num_classes', type='Parameter', properties={}), Node(id='num_classes=1000', type='Parameter', properties={}), Node(id='number of classes to predict with this model', type='Definition', properties={}), Node(id='Neural network model', type='Definition', properties={}), Node(id='layers proposed by AlexNet paper', type='Paper', properties={}), Node(id='(b x 3 x 227 x 227)', type='Input Size', properties={}), Node(id='AlexNet', type='Model', properties={})]
Relationships:[Relationship(source=Node(id='AlexNet', type='Model', properties={}), target=Node(id='layers proposed by AlexNet paper', type='Paper', properties={}), type='CONSISTS_OF', properties={}), Relationship(source=Node(id='AlexNet', type='Model', properties={}), target=Node(id='Neural network model', type='Definition', properties={}), type='DEFINITION', properties={}), Relationship(source=Node(id='AlexNet', type='Model', properties={}), target=Node(id='num_classes=1000', type='Parameter', properties={}), type='PARA

In [ ]:
llm_transformer_filtered = LLMGraphTransformer(
    llm = llm,
    allowed_nodes =["Classes", "", "Parameters"],
    # allowed_relationships=[""]
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

Nodes:[Node(id='layers propsed by AlexNet paper', type='Layers', properties={}), Node(id='AlexNet', type='Model', properties={})]
Relationships:[Relationship(source=Node(id='AlexNet', type='Model', properties={}), target=Node(id='layers propsed by AlexNet paper', type='Layers', properties={}), type='CONSISTS_OF', properties={})]


In [3]:
def query_ollama(prompt: str) -> str:
    response = llm.chat(messages=[{"role": "user", "content": prompt}])
    return response["text"]

In [ ]:
chain = GraphQAChain.from_llm(llm = query_ollama, graph=graph, verbose=True)

In [ ]:
graph_qa_chain = GraphQAChain(
    graph=NetworkxEntityGraph(graph),
    llm=query_ollama
)

In [ ]:
query = ""
result = graph_qa_chain.run(query)
print(result)